# 2048 DQN 모델 학습

## 학습 계획
- **목적**: 2048 게임에서 고득점을 달성하는 DQN 에이전트 학습
- **아키텍처**: CNN (Layered) vs DNN (Flat) 비교
- **환경**: Google Colab GPU 사용

## 환경 설정

In [ ]:
# 필요한 패키지 설치
!pip install torch torchvision gym matplotlib seaborn tensorboard
!pip install onnx onnxruntime

import torch
print(f"PyTorch 버전: {torch.__version__}")
print(f"CUDA 사용 가능: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name()}")

## 코드 업로드 및 설정

In [ ]:
# Google Drive 마운트
from google.colab import drive
drive.mount('/content/drive')

import sys
import os

# 프로젝트 경로 설정
PROJECT_ROOT = '/content/drive/MyDrive/2048-rl-project'
TRAINING_PATH = os.path.join(PROJECT_ROOT, 'training')

# Python 경로에 추가
if PROJECT_ROOT not in sys.path:
    sys.path.insert(0, PROJECT_ROOT)
if TRAINING_PATH not in sys.path:
    sys.path.insert(0, TRAINING_PATH)

# 작업 디렉토리 변경
os.chdir(TRAINING_PATH)

# __init__.py 파일들이 존재하는지 확인
init_files = [
    os.path.join(PROJECT_ROOT, '__init__.py'),
    os.path.join(TRAINING_PATH, '__init__.py'),
    os.path.join(TRAINING_PATH, 'models', '__init__.py'),
    os.path.join(TRAINING_PATH, 'environment', '__init__.py'),
    os.path.join(TRAINING_PATH, 'train', '__init__.py')
]

print("📁 __init__.py 파일 확인:")
for init_file in init_files:
    exists = "✅" if os.path.exists(init_file) else "❌"
    print(f"  {exists} {init_file}")

print(f"\n✅ 경로 설정 완료!")
print(f"  프로젝트 루트: {PROJECT_ROOT}")
print(f"  학습 경로: {TRAINING_PATH}")
print(f"  현재 작업 디렉토리: {os.getcwd()}")

## 모델 및 환경 불러오기

In [ ]:
# 모듈 import 및 테스트
print("🔄 모듈 로딩 시작...")

try:
    from environment.game_2048 import Game2048Env
    print("✅ Game2048Env 로드 성공")
except ImportError as e:
    print(f"❌ Game2048Env 로드 실패: {e}")

try:
    from models.dqn_agent import DQNAgent
    print("✅ DQNAgent 로드 성공")
except ImportError as e:
    print(f"❌ DQNAgent 로드 실패: {e}")

try:
    from models.networks import count_parameters
    print("✅ Networks 로드 성공")
except ImportError as e:
    print(f"❌ Networks 로드 실패: {e}")

# 기타 필요한 라이브러리
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import deque
import time
from IPython.display import clear_output, display
import pandas as pd

# 시각화 설정
plt.style.use('default')
sns.set_palette("husl")

# 간단한 동작 테스트
print("\n🧪 동작 테스트:")
try:
    # 환경 생성 테스트
    test_env = Game2048Env(observation_type='flat')
    print(f"✅ 환경 생성 테스트 성공: {test_env.observation_space}")
    
    # 에이전트 생성 테스트
    test_agent = DQNAgent(observation_type='flat', buffer_size=1000, batch_size=32)
    print(f"✅ 에이전트 생성 테스트 성공: {count_parameters(test_agent.q_network):,} 파라미터")
    
    print("\n🎉 모든 모듈 로드 및 테스트 완료!")
    
except Exception as e:
    print(f"❌ 테스트 실패: {e}")
    print("📋 문제 해결을 위해 다음을 확인하세요:")
    print("  1. 모든 파일이 올바른 경로에 있는지")
    print("  2. __init__.py 파일들이 모두 생성되었는지")
    print("  3. import 경로가 올바른지")

## 학습 설정

In [ ]:
# 학습 하이퍼파라미터
TRAINING_CONFIG = {
    # 기본 설정
    'episodes': 2000,
    'max_steps_per_episode': 1000,
    'device': 'cuda' if torch.cuda.is_available() else 'cpu',
    
    # DQN 설정
    'buffer_size': 100000,
    'batch_size': 64,
    'lr': 1e-4,
    'gamma': 0.99,
    'epsilon_start': 1.0,
    'epsilon_end': 0.01,
    'epsilon_decay': 50000,
    'target_update': 1000,
    
    # 평가 설정
    'eval_interval': 100,
    'eval_episodes': 10,
    'save_interval': 500,
    
    # 시각화 설정
    'plot_interval': 50,
    'log_interval': 10
}

print("⚙️ 학습 설정 완료:")
for key, value in TRAINING_CONFIG.items():
    print(f"  {key}: {value}")

## 모델 초기화

In [ ]:
def create_agent_and_env(observation_type='layered'):
    """에이전트와 환경 생성"""
    env = Game2048Env(observation_type=observation_type)
    
    agent = DQNAgent(
        observation_type=observation_type,
        lr=TRAINING_CONFIG['lr'],
        gamma=TRAINING_CONFIG['gamma'],
        epsilon_start=TRAINING_CONFIG['epsilon_start'],
        epsilon_end=TRAINING_CONFIG['epsilon_end'],
        epsilon_decay=TRAINING_CONFIG['epsilon_decay'],
        buffer_size=TRAINING_CONFIG['buffer_size'],
        batch_size=TRAINING_CONFIG['batch_size'],
        target_update=TRAINING_CONFIG['target_update'],
        device=TRAINING_CONFIG['device']
    )
    
    return agent, env

# CNN과 DNN 에이전트 생성
print("🧠 CNN 에이전트 (Layered) 생성...")
cnn_agent, cnn_env = create_agent_and_env('layered')

print("\n🧠 DNN 에이전트 (Flat) 생성...")
dnn_agent, dnn_env = create_agent_and_env('flat')

print(f"\n📊 모델 비교:")
print(f"  CNN 파라미터: {count_parameters(cnn_agent.q_network):,}")
print(f"  DNN 파라미터: {count_parameters(dnn_agent.q_network):,}")

## 학습 모니터링 함수

In [ ]:
class TrainingMonitor:
    def __init__(self):
        self.reset()
    
    def reset(self):
        self.episode_rewards = []
        self.episode_scores = []
        self.episode_steps = []
        self.episode_losses = []
        self.highest_tiles = []
        self.eval_scores = []
        self.eval_episodes = []
    
    def add_episode(self, reward, score, steps, loss, highest_tile):
        self.episode_rewards.append(reward)
        self.episode_scores.append(score)
        self.episode_steps.append(steps)
        self.episode_losses.append(loss)
        self.highest_tiles.append(highest_tile)
    
    def add_eval(self, episode, avg_score):
        self.eval_episodes.append(episode)
        self.eval_scores.append(avg_score)
    
    def plot_progress(self, title="Training Progress"):
        if len(self.episode_rewards) < 10:
            return
        
        fig, axes = plt.subplots(2, 3, figsize=(18, 12))
        fig.suptitle(title, fontsize=16)
        
        # 보상
        axes[0,0].plot(self.episode_rewards, alpha=0.3, color='blue')
        axes[0,0].plot(pd.Series(self.episode_rewards).rolling(50).mean(), color='red')
        axes[0,0].set_title('Episode Rewards')
        axes[0,0].set_ylabel('Reward')
        
        # 점수
        axes[0,1].plot(self.episode_scores, alpha=0.3, color='green')
        axes[0,1].plot(pd.Series(self.episode_scores).rolling(50).mean(), color='red')
        axes[0,1].set_title('Episode Scores')
        axes[0,1].set_ylabel('Score')
        
        # 최고 타일
        axes[0,2].plot(self.highest_tiles, alpha=0.3, color='purple')
        axes[0,2].plot(pd.Series(self.highest_tiles).rolling(50).mean(), color='red')
        axes[0,2].set_title('Highest Tiles')
        axes[0,2].set_ylabel('Tile Value')
        
        # 스텝 수
        axes[1,0].plot(self.episode_steps, alpha=0.3, color='orange')
        axes[1,0].plot(pd.Series(self.episode_steps).rolling(50).mean(), color='red')
        axes[1,0].set_title('Episode Steps')
        axes[1,0].set_ylabel('Steps')
        axes[1,0].set_xlabel('Episode')
        
        # 손실
        if self.episode_losses and any(loss is not None for loss in self.episode_losses):
            valid_losses = [l for l in self.episode_losses if l is not None]
            if valid_losses:
                axes[1,1].plot(valid_losses, alpha=0.3, color='red')
                axes[1,1].plot(pd.Series(valid_losses).rolling(20).mean(), color='darkred')
        axes[1,1].set_title('Training Loss')
        axes[1,1].set_ylabel('Loss')
        axes[1,1].set_xlabel('Episode')
        
        # 평가 점수
        if self.eval_scores:
            axes[1,2].plot(self.eval_episodes, self.eval_scores, 'o-', color='darkgreen')
        axes[1,2].set_title('Evaluation Scores')
        axes[1,2].set_ylabel('Avg Score')
        axes[1,2].set_xlabel('Episode')
        
        plt.tight_layout()
        plt.show()
    
    def get_stats(self):
        if not self.episode_scores:
            return {}
        
        recent_scores = self.episode_scores[-100:] if len(self.episode_scores) >= 100 else self.episode_scores
        recent_tiles = self.highest_tiles[-100:] if len(self.highest_tiles) >= 100 else self.highest_tiles
        
        return {
            'episodes': len(self.episode_scores),
            'avg_score': np.mean(recent_scores),
            'max_score': max(self.episode_scores),
            'avg_highest_tile': np.mean(recent_tiles),
            'max_highest_tile': max(self.highest_tiles),
            'avg_steps': np.mean(self.episode_steps[-100:]) if len(self.episode_steps) >= 100 else np.mean(self.episode_steps)
        }

# 모니터 초기화
cnn_monitor = TrainingMonitor()
dnn_monitor = TrainingMonitor()

print("📊 학습 모니터링 시스템 준비 완료!")

## 학습 함수

In [ ]:
def train_agent(agent, env, monitor, name, episodes):
    """에이전트 학습 함수"""
    print(f"🚀 {name} 학습 시작! (목표: {episodes} 에피소드)")
    
    start_time = time.time()
    best_score = 0
    
    for episode in range(episodes):
        state = env.reset()
        total_reward = 0
        steps = 0
        episode_losses = []
        
        while steps < TRAINING_CONFIG['max_steps_per_episode']:
            action = agent.select_action(state, training=True)
            next_state, reward, done, info = env.step(action)
            
            agent.store_experience(state, action, reward, next_state, done)
            
            if len(agent.memory) >= agent.batch_size:
                loss = agent.train_step()
                if loss is not None:
                    episode_losses.append(loss)
            
            state = next_state
            total_reward += reward
            steps += 1
            
            if done:
                break
        
        # 에피소드 통계 기록
        avg_loss = np.mean(episode_losses) if episode_losses else None
        monitor.add_episode(total_reward, info['score'], steps, avg_loss, info['highest'])
        agent.episode_rewards.append(total_reward)
        
        # 베스트 모델 저장
        if info['score'] > best_score:
            best_score = info['score']
            try:
                agent.save_model(f'/content/drive/MyDrive/2048_models/{name}_best.pth')
            except Exception as save_error:
                print(f"⚠️ 모델 저장 실패: {save_error}")
        
        # 로그 출력 - 수정된 부분
        if (episode + 1) % TRAINING_CONFIG['log_interval'] == 0:
            try:
                stats = agent.get_stats()
                elapsed = time.time() - start_time
                
                # 손실 값을 미리 포맷팅 (안전한 방법)
                if avg_loss is not None:
                    loss_str = f"{avg_loss:.3f}"
                else:
                    loss_str = "N/A"
                
                print(f"{name} Episode {episode+1:4d} | "
                      f"Score: {info['score']:4.0f} | "
                      f"Reward: {total_reward:6.1f} | "
                      f"Steps: {steps:3d} | "
                      f"Epsilon: {stats['epsilon']:.3f} | "
                      f"Loss: {loss_str} | "
                      f"Time: {elapsed/60:.1f}min")
                      
            except Exception as log_error:
                print(f"⚠️ 로그 출력 에러: {log_error}")
        
        # 평가 및 시각화
        if (episode + 1) % TRAINING_CONFIG['eval_interval'] == 0:
            try:
                eval_score = evaluate_agent(agent, env, TRAINING_CONFIG['eval_episodes'])
                monitor.add_eval(episode + 1, eval_score)
                print(f"🎯 {name} Evaluation (Episode {episode+1}): {eval_score:.1f}")
            except Exception as eval_error:
                print(f"⚠️ 평가 에러: {eval_error}")
        
        if (episode + 1) % TRAINING_CONFIG['plot_interval'] == 0:
            try:
                clear_output(wait=True)
                monitor.plot_progress(f"{name} Training Progress")
                
                stats = monitor.get_stats()
                print(f"\n📊 {name} 현재 통계:")
                
                # 안전한 통계 출력 - 수정된 부분
                for key, value in stats.items():
                    if isinstance(value, float):
                        print(f"  {key}: {value:.2f}")
                    elif isinstance(value, int):
                        print(f"  {key}: {value}")
                    else:
                        print(f"  {key}: {value}")
                        
            except Exception as plot_error:
                print(f"⚠️ 시각화 에러: {plot_error}")
        
        # 정기 저장
        if (episode + 1) % TRAINING_CONFIG['save_interval'] == 0:
            try:
                agent.save_model(f'/content/drive/MyDrive/2048_models/{name}_checkpoint_{episode+1}.pth')
            except Exception as checkpoint_error:
                print(f"⚠️ 체크포인트 저장 실패: {checkpoint_error}")
    
    # 최종 저장
    try:
        agent.save_model(f'/content/drive/MyDrive/2048_models/{name}_final.pth')
        print(f"✅ {name} 학습 완료!")
    except Exception as final_save_error:
        print(f"⚠️ 최종 모델 저장 실패: {final_save_error}")
        print(f"✅ {name} 학습 완료! (저장 실패)")
    
    return monitor

print("🔧 학습 함수 로드 완료!")

## CNN 모델 학습

In [ ]:
# CNN 학습 실행
cnn_monitor = train_agent(
    agent=cnn_agent,
    env=cnn_env,
    monitor=cnn_monitor,
    name="CNN",
    episodes=TRAINING_CONFIG['episodes']
)

## DNN 모델 학습

In [ ]:
# DNN 학습 실행
dnn_monitor = train_agent(
    agent=dnn_agent,
    env=dnn_env,
    monitor=dnn_monitor,
    name="DNN",
    episodes=TRAINING_CONFIG['episodes']
)

## 🏆 최종 성능 비교

In [ ]:
# 최종 평가
print("🏆 최종 성능 평가 (각 50 게임)")

cnn_final_score = evaluate_agent(cnn_agent, cnn_env, 50)
dnn_final_score = evaluate_agent(dnn_agent, dnn_env, 50)

print(f"\n📊 최종 결과:")
print(f"  CNN 평균 점수: {cnn_final_score:.1f}")
print(f"  DNN 평균 점수: {dnn_final_score:.1f}")

winner = "CNN" if cnn_final_score > dnn_final_score else "DNN"
diff = abs(cnn_final_score - dnn_final_score)
print(f"\n🏆 승자: {winner} (평균 {diff:.1f}점 차이)")

# 통계 비교
cnn_stats = cnn_monitor.get_stats()
dnn_stats = dnn_monitor.get_stats()

comparison_df = pd.DataFrame({
    'CNN': [cnn_stats.get(k, 0) for k in ['avg_score', 'max_score', 'max_highest_tile', 'avg_steps']],
    'DNN': [dnn_stats.get(k, 0) for k in ['avg_score', 'max_score', 'max_highest_tile', 'avg_steps']]
}, index=['평균 점수', '최고 점수', '최고 타일', '평균 스텝'])

print("\n📈 상세 비교:")
print(comparison_df)

# 최종 시각화
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# CNN 결과
axes[0].plot(cnn_monitor.episode_scores, alpha=0.3, label='Episode Scores')
axes[0].plot(pd.Series(cnn_monitor.episode_scores).rolling(100).mean(), label='Moving Average (100)')
axes[0].set_title('CNN Training Progress')
axes[0].set_xlabel('Episode')
axes[0].set_ylabel('Score')
axes[0].legend()

# DNN 결과
axes[1].plot(dnn_monitor.episode_scores, alpha=0.3, label='Episode Scores')
axes[1].plot(pd.Series(dnn_monitor.episode_scores).rolling(100).mean(), label='Moving Average (100)')
axes[1].set_title('DNN Training Progress')
axes[1].set_xlabel('Episode')
axes[1].set_ylabel('Score')
axes[1].legend()

plt.tight_layout()
plt.show()

## ONNX 변환

In [ ]:
# 베스트 모델 ONNX 변환
print("🔄 ONNX 변환 시작...")

# CNN 모델 변환
cnn_agent.export_to_onnx(
    filepath='/content/drive/MyDrive/2048_models/cnn_model.onnx',
    input_shape=(4, 4, 16)
)

# DNN 모델 변환
dnn_agent.export_to_onnx(
    filepath='/content/drive/MyDrive/2048_models/dnn_model.onnx',
    input_shape=(16,)
)

print("✅ ONNX 변환 완료!")
print("  - CNN: /content/drive/MyDrive/2048_models/cnn_model.onnx")
print("  - DNN: /content/drive/MyDrive/2048_models/dnn_model.onnx")

## 데모 플레이

In [ ]:
# 학습된 모델로 게임 플레이 데모
def demo_play(agent, env, name, render_interval=10):
    print(f"🎮 {name} 데모 플레이 시작!")
    
    state = env.reset()
    done = False
    steps = 0
    
    while not done and steps < 1000:
        if steps % render_interval == 0:
            clear_output(wait=True)
            env.render()
            print(f"Step: {steps}")
            time.sleep(0.5)
        
        action = agent.select_action(state, training=False)
        state, reward, done, info = env.step(action)
        steps += 1
    
    env.render()
    print(f"\n🏁 {name} 게임 종료!")
    print(f"  최종 점수: {info['score']}")
    print(f"  최고 타일: {info['highest']}")
    print(f"  총 스텝: {steps}")

# 베스트 모델로 데모
best_agent = cnn_agent if cnn_final_score > dnn_final_score else dnn_agent
best_env = cnn_env if cnn_final_score > dnn_final_score else dnn_env
best_name = "CNN" if cnn_final_score > dnn_final_score else "DNN"

demo_play(best_agent, best_env, f"Best {best_name}")